In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

In [2]:
llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a+b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(func=plus, name="Sum Calculator", description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers.")
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all these amounts added together.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:I have calculated the sum of the first two amounts. Now, I will continue to calculate the total cost by adding the remaining amounts using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three amounts. Now, I will continue to calculate the total cost by adding the remaining amounts using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:I have calculated the sum of the 

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost is $4616.1.'}